***Digital Mirror***

*This is a personal philosophy and growth mirror*


In [1]:
import sqlite3
import json
import os
from dotenv import load_dotenv
from openai import OpenAI
from ddgs import DDGS
import gradio as gr

def initialize_db():
    conn = sqlite3.connect('digital_mirror.db')
    cursor = conn.cursor()

    #This stores the user's state of beliefs
    cursor.execute('''CREATE TABLE IF NOT EXISTS beliefs 
                     (topic TEXT PRIMARY KEY, stance TEXT, last_updated DATETIME)''')

    #This stores the conversation history
    cursor.execute('''CREATE TABLE IF NOT EXISTS logs 
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      role TEXT,
                      content TEXT,
                      timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')

    conn.commit()
    conn.close()

def get_belief(topic):
    conn = sqlite3.connect('digital_mirror.db')
    cursor = conn.cursor()
    cursor.execute("SELECT stance FROM beliefs WHERE topic = ?", (topic,))
    belief = cursor.fetchone()
    conn.close()
    return belief[0] if belief else None

def update_belief(topic, stance):
    conn = sqlite3.connect('digital_mirror.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT OR REPLACE INTO beliefs (topic, stance, last_updated)
                         VALUES (?, ?, CURRENT_TIMESTAMP)''', (topic, stance))
    conn.commit()
    conn.close()

def log_message(role, content):
    conn = sqlite3.connect('digital_mirror.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO logs (role, content) VALUES (?, ?)", (role, content))
    conn.commit()
    conn.close()

def fetch_beliefs():
    conn = sqlite3.connect('digital_mirror.db')
    cursor = conn.cursor()
    cursor.execute("SELECT topic, stance, last_updated FROM beliefs")
    data = cursor.fetchall()
    conn.close()
    return data
        
        
        


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openai_api_key
)

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


OpenAI API Key exists and begins sk-or-v1


In [3]:
def update_personal_beliefs(topic, user_input, current_summary):

    prompt = f"""
    Current stored belief: {current_summary}
    New user input: {user_input}
    
    Has the user's stance evolved, stayed the same, or contradicted itself?

    Output JSON:
    {{
      "status": "evolved/same/contradicted",
      "new_stance": "short summarized stance"
    }}
    """

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    result = json.loads(response.choices[0].message.content)

    if result["status"] in ["evolved", "contradicted"]:
        update_belief(topic, result["new_stance"])

    return result


In [4]:
system_prompt = """
You are a tough philosopher.

Your role:
- Find the strongest counter-argument to the user's stance
- If they contradict themselves, point it out
- If consistent, reinforce their stance
- Frame critique as a stress test for growth, not a personal attack
- End with a deep reflection question
"""

def web_counter_argument(topic):
    with DDGS() as ddgs:
        results = list(ddgs.text(topic, max_results=3))
        if results:
            return results[0]["body"]
        return "No external counter-argument found."

def tough_response(topic, user_input):

    past_belief = get_belief(topic)
    counter_argument = web_counter_argument(topic)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"""
        Topic: {topic}
        Past belief: {past_belief}
        Current position: {user_input}
        Counter-argument: {counter_argument}
        """}
    ]

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
        temperature=0.8
    )

    return response.choices[0].message.content


In [5]:
initialize_db()

def chat(topic, user_input, history):

    if history is None:
        history = []

    # Log user
    log_message("user", user_input)

    past = get_belief(topic)
    update_personal_beliefs(topic, user_input, past)

    reply = tough_response(topic, user_input)

    # log assistant
    log_message("assistant", reply)

    # proper Chatbot format
    history.append({"role": "user", "content": user_input})
    history.append({"role": "assistant", "content": reply})

    table_data = fetch_beliefs()

    return history, table_data



In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("Digital Mirror")

    with gr.Row():
        with gr.Column():
            topic = gr.Textbox(label="Topic")
            message = gr.Textbox(label="Your Position")
            chatbot = gr.Chatbot()
            send = gr.Button("Send")

        with gr.Column():
            belief_table = gr.Dataframe(
                headers=["Topic", "Stance", "Last Updated"],
                interactive=False
            )

    send.click(
        chat,
        inputs=[topic, message, chatbot],
        outputs=[chatbot, belief_table]
    )

demo.launch()